In [1]:
!pip install beir

     |████████████████████████████████| 52 kB 1.6 MB/s 
     |████████████████████████████████| 78 kB 5.7 MB/s 
     |████████████████████████████████| 8.4 MB 23.1 MB/s 
     |████████████████████████████████| 379 kB 78.4 MB/s 
     |████████████████████████████████| 4.9 MB 58.0 MB/s 
     |████████████████████████████████| 3.1 MB 51.6 MB/s 
     |████████████████████████████████| 3.3 MB 76.2 MB/s 
     |████████████████████████████████| 1.2 MB 71.3 MB/s 
     |████████████████████████████████| 59 kB 6.6 MB/s 
     |████████████████████████████████| 596 kB 61.4 MB/s 
     |████████████████████████████████| 895 kB 66.7 MB/s 
  Created wheel for beir: filename=beir-0.2.3-py3-none-any.whl size=45805 sha256=40a5f3991b39a2ce6fcf5718046ee8d0fb3782f3ecd4f52eba0c57e3ac94bbf9
  Stored in directory: /root/.cache/pip/wheels/af/29/4c/261c842753d34921b8b35c1b8679720114f8536654767430dd
  Created wheel for pytrec-eval: filename=pytrec_eval-0.5-cp37-cp37m-linux_x86_64.whl size=265984 sha256=98ae934372

In [2]:
!pip install sentence_transformers

In [3]:
!pip install datasets

     |████████████████████████████████| 290 kB 7.3 MB/s 
     |████████████████████████████████| 132 kB 87.6 MB/s 
     |████████████████████████████████| 1.1 MB 66.5 MB/s 
     |████████████████████████████████| 243 kB 90.5 MB/s 
     |████████████████████████████████| 271 kB 80.1 MB/s 
     |████████████████████████████████| 192 kB 74.3 MB/s 
     |████████████████████████████████| 160 kB 89.7 MB/s 


In [4]:
!pip install hnswlib

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for hnswlib: filename=hnswlib-0.5.2-cp37-cp37m-linux_x86_64.whl size=1326164 sha256=634cd53460402250b9fcd01a06a75691ad0beef380915195e7afd72db849a4e8
  Stored in directory: /root/.cache/pip/wheels/b4/11/b3/337c4a361b31217d62c3b420ad66fe20d381f1ebb29b046095
Successfully built hnswlib


In [5]:
import pandas as pd
import numpy as np

from beir import util
from beir.datasets.data_loader import GenericDataLoader

from datasets import load_dataset, load_metric

from typing import List
from typing import Dict
from typing import Tuple

from sentence_transformers import SentenceTransformer
import sentence_transformers.util

import hnswlib
import pickle

from operator import itemgetter

import time

/usr/local/lib/python3.7/dist-packages/beir/util.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [6]:
def create_the_whole_context_dataset(model_name):

    # Load squadv2
    squad_v2 = False
    train_dataset = load_dataset("squad_v2" if squad_v2 else "squad", split='train[:10%]')

    # Load DBpedia
    dataset = "dbpedia-entity"
    url = "https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/dbpedia-entity.zip".format(dataset)
    data_path = util.download_and_unzip(url, "datasets")
    corpus, queries, qrels = GenericDataLoader(data_folder=data_path).load(split="test")

    # Create the list of all context
    list_context = []

    ## Add from squad_v2
    for elem in train_dataset:
        context = elem["context"]
        if (not context in list_context):
            list_context.append(context)

    ## Add from dbpedia
    corpus_list = list(corpus)
    random_element = np.random.choice(corpus_list, 10000 - len(list_context) + 1500)

    random_element = [corpus[elem]["text"] for elem in random_element]

    random_element = [elem for elem in random_element if len(elem) >= 50]

    random_element = np.unique(random_element)

    for elem in random_element:
        if (not elem in list_context):
            list_context.append(elem)


    # Create the list with formatedcontext
    model = SentenceTransformer(model_name)

    list_formated_context = model.encode(list_context, device='cuda', show_progress_bar=True)
    list_formated_context = [[list_context[i], list_formated_context[i], i] for i in range(len(list_formated_context))]

    return list_formated_context

In [7]:
def create_and_train_model(param1, param2, context_list):

    dim = len(context_list[0][1])
    num_elements = len(context_list)

    p = hnswlib.Index(space = 'ip', dim = dim)
    p.init_index(max_elements = num_elements, ef_construction = param1, M = param2)

    datas = [elem[1] for elem in context_list]
    indexs = [elem[2] for elem in context_list]

    p.add_items(datas, indexs)

    return p

In [8]:
def find_best_context(model_name, knn, question, n, formated_context):

    model = SentenceTransformer(model_name)
    formated_question = model.encode(question, device='cuda', show_progress_bar=False)

    labels, distances = knn.knn_query(formated_question, k = n)

    context_list = []

    for label in labels[0]:
      context_list.append(formated_context[label][0])

    return context_list